In [1]:
import os
import glob
import pickle
import copy
import pprint
from peewee import *
from playhouse.apsw_ext import APSWDatabase
from playhouse.migrate import SqliteMigrator, migrate

In [2]:
paths = glob.glob("/Users/Venky/Work/UIowa/DEEPN/dictionaries/*.p")

# Database Definition

In [3]:
database = APSWDatabase(None, pragmas={'journal_mode': 'off',
                                       'cache_size': -500*1000,
                                       'ignore_check_constraints': 0,
                                       'synchronous': 0,
                                       'foreign_keys': 1,
                                       'temp_store': 'memory',
                                       'locking_mode': 'exclusive'}) 

In [4]:
def create_class(class_name, parent=Model):
    NewClass = type(class_name, (parent,), {
        "start": IntegerField(),
        "stop": IntegerField(),
        "chromosome_name": CharField(),
        "gene_name": CharField(),
        "exontype": BooleanField(),
    })
    NewClass._meta.database = database
    NewClass._meta.table_name = class_name
    return NewClass

In [5]:
database.init("/Users/Venky/Work/UIowa/DEEPN/dictionaries/exons.db")
for path in paths:
    table = create_class(os.path.basename(os.path.basename(path).replace(".p", "")))
    print "Creating table: ", table._meta.table_name
    database.create_tables([table])
    exon_list = pickle.load(open(path, "rb"))
    for ch in exon_list.keys():
        print "Chromosome: ", ch
        for exon in exon_list[ch].keys():
            if not table.select().where(table.start==exon[0], table.stop==exon[1], table.chromosome_name==exon[2], table.gene_name==exon[4], table.exontype==True if exon[-1] == 'Y' else False):
                table.insert(start=exon[0], stop=exon[1], chromosome_name=exon[2], gene_name=exon[4], exontype=True if exon[-1] == 'Y' else False).execute()
database.close()

Creating table:  hg38exonDict
Chromosome:  19_KI270882v1_alt
Chromosome:  15_KI270849v1_alt
Chromosome:  22_KI270879v1_alt
Chromosome:  10_KI270825v1_alt
Chromosome:  7_KI270804v1_alt
Chromosome:  3_KI270937v1_alt
Chromosome:  1_KI270762v1_alt
Chromosome:  6_GL000253v2_alt
Chromosome:  18_GL383571v1_alt
Chromosome:  17_GL383563v3_alt
Chromosome:  19_GL949753v2_alt
Chromosome:  15_KI270850v1_alt
Chromosome:  17_KI270908v1_alt
Chromosome:  20
Chromosome:  21
Chromosome:  22
Chromosome:  17_GL383564v2_alt
Chromosome:  21_KI270873v1_alt
Chromosome:  Un_GL000213v1
Chromosome:  8_KI270821v1_alt
Chromosome:  8_KI270819v1_alt
Chromosome:  4
Chromosome:  11_KI270831v1_alt
Chromosome:  19_KI270867v1_alt
Chromosome:  15_GL383554v1_alt
Chromosome:  8
Chromosome:  11_KI270927v1_alt
Chromosome:  12_GL877876v1_alt
Chromosome:  11
Chromosome:  1_KI270759v1_alt
Chromosome:  7
Chromosome:  19_KI270930v1_alt
Chromosome:  19_KI270868v1_alt
Chromosome:  13_KI270842v1_alt
Chromosome:  3_KI270924v1_alt
Chrom

True